In [16]:
import os
import csv
import json
import pandas as pd
import pickle
import plotly

In [20]:
DATALOG_FILENAME_BASE = 'datalog2'
DATALOG_FILENAME_FULL = DATALOG_FILENAME_BASE + '.csv'
DATALOG_PICKLE_NAME = DATALOG_FILENAME_BASE + '.pkl'

# file -I datalog2.csv reveals:
#    datalog2.csv: text/csv; charset=iso-8859-1
#    - this must be a COBB AP Spec, so the program handles it
DATALOG_ENCODING = 'iso-8859-1'

RAW_DATA_FRAME = None

In [60]:
try:
    with open(DATALOG_PICKLE_NAME, 'rb') as log_pkl:
        RAW_DATA_FRAME = pickle.load(log_pkl)
except:
    with open(DATALOG_FILENAME_FULL, 'r', encoding=DATALOG_ENCODING) as log_file:
        RAW_DATA_FRAME = pd.read_csv(log_file, index_col=0)
    
    with open(DATALOG_FILENAME_BASE + '.pkl', 'wb+') as log_pkl:
        pickle.dump(RAW_DATA_FRAME, log_pkl)

df = RAW_DATA_FRAME

In [61]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Float64Index: 11133 entries, 0.0 to 1970.408
Data columns (total 38 columns):
 #   Column                                                                                                                                                                        Non-Null Count  Dtype  
---  ------                                                                                                                                                                        --------------  -----  
 0   AF Correction 1 (%)                                                                                                                                                           11133 non-null  float64
 1   AF Correction 3 (%)                                                                                                                                                           11133 non-null  float64
 2   AF Learning 1 (%)                                                  

In [64]:
try:
    INFO = df.columns[-1]
    if INFO.startswith('AP Info'):
        df = df.drop(columns=INFO)
except: 
    pass

print(INFO)
print(df.columns)

Wastegate Duty (%)
Index(['AF Correction 1 (%)', 'AF Correction 3 (%)', 'AF Learning 1 (%)',
       'AF Sens 1 Ratio (AFR)', 'AF Sens 3 Volts (V)', 'Accel Position (%)',
       'Baro Pressure (psi)', 'Boost (psi)', 'Calculated Load (g/rev)',
       'Clutch Sw (on/off)', 'Comm Fuel Final (AFR)', 'Coolant Temp (F)',
       'Dyn Adv Mult (DAM)', 'Dynamic Adv Lrn (°)', 'Feedback Knock (°)',
       'Fine Knock Learn (°)', 'Fuel Economy (mpg)', 'Fuel Pump Duty (%)',
       'Gear Position (Gear)', 'Idle Airflow (g/s)', 'Idle Spd Target (RPM)',
       'Idle Speed Error (RPM)', 'Ignition Timing (°)', 'Inj Duty Cycle (%)',
       'Intake Temp (F)', 'MAF (g/s)', 'MAF Volts (V)', 'Man Abs Press (psi)',
       'RPM (RPM)', 'RPM Delta (RPM)', 'Req Torque (Nm)',
       'SI Drive Mode (raw)', 'TD Boost Error (psi)',
       'Target Boost Final Rel SL (psi)', 'Throttle Pos (%)',
       'Vehicle Speed (mph)', 'Wastegate Duty (%)'],
      dtype='object')
